In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Conv2D, MaxPooling2D, Flatten, Dense, LSTM, concatenate, Dropout, BatchNormalization, Activation
from tensorflow.keras.utils import plot_model
from tensorflow.keras.wrappers.scikit_learn import KerasClassifier
from sklearn.model_selection import RandomizedSearchCV
from tensorflow.keras.callbacks import ModelCheckpoint

# seed
#import os
#seed = 123
#os.environ['PYTHONHASHSEED'] = str(seed)
#np.random.seed(seed)
#tf.random.set_seed(seed)

In [2]:
train = pd.read_csv('data/train.csv')
test = pd.read_csv('data/test.csv')

In [3]:
x = train.drop(['id', 'digit', 'letter'], axis=1).values
x = x.reshape(-1, 28, 28, 1)
x = x/255

y_data = train['digit']
y = np.zeros((len(y_data), len(y_data.unique())))
for i, digit in enumerate(y_data):
    y[i, digit] = 1

In [4]:
x_letter = train['letter'].values
x_letter = x_letter[:, np.newaxis]
en = OneHotEncoder()
x_letter = en.fit_transform(x_letter).toarray()
x_letter.shape

(2048, 26)

In [5]:
x_train, x_val, y_train, y_val = train_test_split(x, y, test_size=0.2, shuffle=True, stratify=y)

print(x_train.shape)
print(x_val.shape)
print(y_train.shape)
print(y_val.shape)

x_letter_train = x_letter[:x_train.shape[0],:]
x_letter_val = x_letter[x_train.shape[0]:,:]
print(x_letter_train.shape)
print(x_letter_val.shape)

(1638, 28, 28, 1)
(410, 28, 28, 1)
(1638, 10)
(410, 10)
(1638, 26)
(410, 26)


In [6]:
input1 = Input(shape=(28,28,1))
x1 = Conv2D(64, (3,3), padding='same', activation='elu')(input1)
#x1 = BatchNormalization()(x1)
#x1 = Activation('elu')(x1)
x1 = Dropout(0.3)(x1)
x1 = MaxPooling2D((2,2))(x1)
x1 = Conv2D(64, (2,2), padding='same', activation='elu')(x1)
#x1 = BatchNormalization()(x1)
#x1 = Activation('elu')(x1)
x1 = Dropout(0.3)(x1)
x1 = MaxPooling2D((2,2))(x1)
x1 = Conv2D(128, (2,2), padding='same', activation='elu')(x1)
#x1 = BatchNormalization()(x1)
#x1 = Activation('elu')(x1)
x1 = Dropout(0.3)(x1)
x1 = MaxPooling2D((2,2))(x1)
x1 = Flatten()(x1)

input2 = Input(shape=(26,))
merge = concatenate([x1, input2])

x2 = Dense(500, activation='relu')(merge)
x2 = Dropout(0.3)(x2)
x2 = Dense(100, activation='relu')(x2)
x2 = Dropout(0.3)(x2)
x2 = Dense(50, activation='relu')(x2)
outputs = Dense(10, activation='softmax')(x2)

model = Model(inputs = [input1, input2], outputs = outputs)

model.summary()

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 28, 28, 1)]  0                                            
__________________________________________________________________________________________________
conv2d (Conv2D)                 (None, 28, 28, 64)   640         input_1[0][0]                    
__________________________________________________________________________________________________
dropout (Dropout)               (None, 28, 28, 64)   0           conv2d[0][0]                     
__________________________________________________________________________________________________
max_pooling2d (MaxPooling2D)    (None, 14, 14, 64)   0           dropout[0][0]                    
______________________________________________________________________________________________

In [7]:
model.compile(optimizer = 'adam', metrics = ['accuracy'], loss = 'categorical_crossentropy')
history = model.fit([x_train, x_letter_train], y_train, validation_data=([x_val, x_letter_val], y_val), 
                    batch_size=64, epochs=100, verbose=1, 
                    callbacks = [ModelCheckpoint('./models/{epoch:02d}-{val_accuracy:.4f}.h5',
                    monitor='val_loss', verbose=1, save_best_only=True, mode='auto')])

Train on 1638 samples, validate on 410 samples
Epoch 1/100
1600/1638 [============================>.] - ETA: 0s - loss: 2.3052 - accuracy: 0.0994
Epoch 00001: val_loss improved from inf to 2.29296, saving model to ./models/01-2.2930.h5
1638/1638 [==============================] - 4s 2ms/sample - loss: 2.3049 - accuracy: 0.1001 - val_loss: 2.2930 - val_accuracy: 0.1537
Epoch 2/100
1600/1638 [============================>.] - ETA: 0s - loss: 2.2784 - accuracy: 0.1350
Epoch 00002: val_loss improved from 2.29296 to 2.23539, saving model to ./models/02-2.2354.h5
1638/1638 [==============================] - 3s 2ms/sample - loss: 2.2788 - accuracy: 0.1337 - val_loss: 2.2354 - val_accuracy: 0.1390
Epoch 3/100
1600/1638 [============================>.] - ETA: 0s - loss: 2.1789 - accuracy: 0.1869
Epoch 00003: val_loss improved from 2.23539 to 2.15975, saving model to ./models/03-2.1598.h5
1638/1638 [==============================] - 3s 2ms/sample - loss: 2.1782 - accuracy: 0.1874 - val_loss: 2.1

Epoch 29/100
1600/1638 [============================>.] - ETA: 0s - loss: 0.3457 - accuracy: 0.8788
Epoch 00029: val_loss did not improve from 1.96847
1638/1638 [==============================] - 3s 2ms/sample - loss: 0.3418 - accuracy: 0.8797 - val_loss: 5.3572 - val_accuracy: 0.3341
Epoch 30/100
1600/1638 [============================>.] - ETA: 0s - loss: 0.2922 - accuracy: 0.9038
Epoch 00030: val_loss did not improve from 1.96847
1638/1638 [==============================] - 3s 2ms/sample - loss: 0.2965 - accuracy: 0.9017 - val_loss: 6.4877 - val_accuracy: 0.3463
Epoch 31/100
1600/1638 [============================>.] - ETA: 0s - loss: 0.2889 - accuracy: 0.8950
Epoch 00031: val_loss did not improve from 1.96847
1638/1638 [==============================] - 3s 2ms/sample - loss: 0.2945 - accuracy: 0.8926 - val_loss: 5.5663 - val_accuracy: 0.3293
Epoch 32/100
1600/1638 [============================>.] - ETA: 0s - loss: 0.2812 - accuracy: 0.9019
Epoch 00032: val_loss did not improve from

Epoch 58/100
1600/1638 [============================>.] - ETA: 0s - loss: 0.1345 - accuracy: 0.9563
Epoch 00058: val_loss did not improve from 1.96847
1638/1638 [==============================] - 3s 2ms/sample - loss: 0.1329 - accuracy: 0.9567 - val_loss: 6.2168 - val_accuracy: 0.3659
Epoch 59/100
1600/1638 [============================>.] - ETA: 0s - loss: 0.0939 - accuracy: 0.9719
Epoch 00059: val_loss did not improve from 1.96847
1638/1638 [==============================] - 3s 2ms/sample - loss: 0.0940 - accuracy: 0.9719 - val_loss: 7.4361 - val_accuracy: 0.3268
Epoch 60/100
1600/1638 [============================>.] - ETA: 0s - loss: 0.1133 - accuracy: 0.9663
Epoch 00060: val_loss did not improve from 1.96847
1638/1638 [==============================] - 3s 2ms/sample - loss: 0.1123 - accuracy: 0.9664 - val_loss: 7.3889 - val_accuracy: 0.3585
Epoch 61/100
1600/1638 [============================>.] - ETA: 0s - loss: 0.0818 - accuracy: 0.9769
Epoch 00061: val_loss did not improve from

Epoch 87/100
1600/1638 [============================>.] - ETA: 0s - loss: 0.0766 - accuracy: 0.9781
Epoch 00087: val_loss did not improve from 1.96847
1638/1638 [==============================] - 3s 2ms/sample - loss: 0.0771 - accuracy: 0.9774 - val_loss: 6.2950 - val_accuracy: 0.4073
Epoch 88/100
1600/1638 [============================>.] - ETA: 0s - loss: 0.0762 - accuracy: 0.9787
Epoch 00088: val_loss did not improve from 1.96847
1638/1638 [==============================] - 3s 2ms/sample - loss: 0.0750 - accuracy: 0.9792 - val_loss: 5.4437 - val_accuracy: 0.3707
Epoch 89/100
1600/1638 [============================>.] - ETA: 0s - loss: 0.0496 - accuracy: 0.9812
Epoch 00089: val_loss did not improve from 1.96847
1638/1638 [==============================] - 3s 2ms/sample - loss: 0.0489 - accuracy: 0.9817 - val_loss: 5.8887 - val_accuracy: 0.3902
Epoch 90/100
1600/1638 [============================>.] - ETA: 0s - loss: 0.0559 - accuracy: 0.9812
Epoch 00090: val_loss did not improve from

In [8]:
x_test = test.drop(['id', 'letter'], axis=1).values
x_test = x_test.reshape(-1, 28, 28, 1)
x_test = x_test/255

x_letter_test = test['letter'].values
x_letter_test = x_letter_test[:, np.newaxis]
en = OneHotEncoder()
x_letter_test = en.fit_transform(x_letter_test).toarray()
x_letter_test.shape

(20480, 26)

In [9]:
#submission = pd.read_csv('data/submission.csv')
#submission['digit'] = np.argmax(model.predict([x_test, x_letter_test]), axis=1)

In [10]:
#submission.to_csv('data/submission1.csv', index=False)